<h3 style="text-align: center;">Monte Carlo (MC) Simulations for 2SLS and two-step efficient GMM</h3>
<h4 style="text-align: center;">Maria Santos</h4>

In [1]:
# packages
using Distributions
using LinearAlgebra
using PrettyTables
using Parameters
using Statistics
using Random 

a) Generate $n=500$ observations for $(Y_i, X_i, Z_i)$ from the following IV regression model with heteroskedastic errors:

$$Y_i=\beta X_i + U_i$$
$$X_i=\beta \pi' Z_i + V_i$$
$$U_i=\beta e^{\gamma' Z_i} \times \varepsilon_i$$

where $Z_i \sim N(0, I_2)$ and is independent of $\varepsilon_i$ and $V_i$, and

$$\left( \begin{array}{c}  \epsilon_i \\ V_i \end{array} \right)  \sim N\left( \left( \begin{array}{c}  0 \\ 0 \end{array} \right), \left( \begin{array}{c}  1 & \rho\\ \rho & 1 \end{array} \right) \right).$$

Use the following values of the parameters:

$\beta = 1$, $\pi = \gamma =  \left( \begin{array}{c}  1 \\ 1 \end{array} \right)$, and  $\rho =0.95$.

In [2]:
function generate_observations(true_β, π, γ, ρ, n, seed)
    Random.seed!(seed)  # Set seed for reproducibility
    
    # Generate Z_i from a multivariate normal distribution with mean 0 and variance I_2
    Z = randn(n, 2)
    
    # Generate ε_i and V_i from a multivariate normal distribution
    Σ = [1 ρ; ρ 1]  # Covariance matrix for ε_i and V_i
    mvnormal = MvNormal([0.0; 0.0], Σ)
    #errors = randn(n, 2) * cholesky(Σ).U  # Generate correlated errors
    errors = rand(mvnormal, n)'  # Generate correlated errors

    # Calculate U_i using the provided equation
    #U = [exp.(dot(row, transpose(γ))) for row in eachrow(Z)] .* errors[:, 1]
    U = exp.(Z * γ) .* errors[:, 1]

    # Calculate X_i using the provided equation
    V = errors[:, 2]
    #X = [dot(row, transpose(π)) for row in eachrow(Z)] + V
    X = Z * π + V

    # Calculate Y_i using the provided equation
    Y = β * X + U
    
    return Y, X, Z
end;


In [3]:
β = 1
π = [1;1]
γ = [1;1]
ρ = 0.95
n = 500
seed = 1995;  # Seed for reproducibility

In [4]:
Y,X,Z = generate_observations(β, π, γ, ρ, n, seed);

b) Compute $\hat{\beta}^{2SLS}$, the 2SLS estimator, and its standard error.

In [ ]:
"""
function ordinary_least_squares(X, y)
    n = length(y)
    beta_hat = (X' * X) \ (X' * y)
    y_pred = X * beta_hat
    
    if isa(X' * X, Matrix) == false
        se = sqrt(X' * X)
    else
        se = sqrt.(diag(X' * X))
    end
    
    return beta_hat, se, y_pred
end;
"""


In [5]:
function two_stage_least_squares(y, X, Z)
    # define the number of observations
    n = length(y)

    # define the projection matrix
    P_z = Z * inv(Z' * Z) * Z'
    P_z_x = P_z * X

    # estimate the 2SLS estimator using the projection matrix
    β_hat = (P_z_x' * P_z_x) \ (P_z_x' * y)

    # calculate the residuals
    residuals = y - P_z_x*β_hat
    rX = P_z_x .* residuals
    
    # estimate the variance of the residuals
    W = Z'*Z / size(Y)[1]
    W2 = (Z .* residuals)' * (Z .* residuals) / n
    Q = (X' * Z) / size(Y)[1]
    
    asymptotic_var = (inv(Q * inv(W) * Q')*(Q * inv(W) * W2 * inv(W) * Q')*inv(Q * inv(W) * Q') / length(Y))

    # calculate the standard errors
    se = sqrt.(asymptotic_var)   

    return β_hat, se
end;


In [6]:
beta_hat, se = two_stage_least_squares(Y, X, Z);

c) Compute $\hat{\beta}^{GMM}$, the two-step GMM estimator of $\beta$, and its standard error.

In [7]:
function gmm_estimator(X, Y, Z, W)
    n = size(X, 1)
    p = size(X, 2)
    q = size(Z, 2)
    
    # Compute the intermediate matrices
    XZ_sum = zeros(p, q)
    ZX_sum = zeros(q, p)
    ZY_sum = zeros(q, 1)
    
    for i in 1:n
        XZ_sum += X[i, :] * Z[i, :]'
        ZX_sum += Z[i, :] * X[i, :]'
        ZY_sum += Z[i, :] * Y[i]
    end
    
    # Compute beta_hat using the formula directly
    β_hat = inv(XZ_sum * inv(W) * ZX_sum) * XZ_sum * inv(W) * ZY_sum
    
    return β_hat
end;

In [8]:
function two_step_gmm(X, Y, Z)
    # 1st step: define identity matrix as W
    W = Matrix(I, size(Z)[2], size(Z)[2])
    
    # 2nd step: estimate beta_hat (inneficient estimator)
    β_hat = gmm_estimator(X, Y, Z, W)

    # 3rd step: estimate omega (weight matrix)
    residuals = Y - X * β_hat
    #W2 = sum(reshape(residuals .^ 2, 1, 1,size(Y)[1]) .* (reshape(Z, size(Z)[2], 1, size(Y)[1]) .* reshape(Z, 1, size(Z)[2], size(Y)[1])), dims=size(Z)[2]+1) / size(X)[1];
    #W2 = reshape(W2, size(Z)[2], size(Z)[2])
    W2 = (Z .* residuals)' * (Z .* residuals) / n    
    
    # 4th step: estimate beta_hat (efficient estimator)
    β_hat2 = gmm_estimator(X, Y, Z, W2)

    # 5th step: estimate omega (weight matrix)
    # standard error
    residuals2 = Y - X * β_hat2
    W3 = (Z .* residuals2)' * (Z .* residuals2) / n
    Q = (X' * Z) / size(Y)[1]
    
    asymptotic_error = sqrt(inv(Q * inv(W2) * Q')*(Q * inv(W2) * W3 * inv(W2) * Q')*inv(Q * inv(W2) * Q') / length(Y))
    
    return β_hat2[1], asymptotic_error
end;

In [9]:
beta_hat, se = two_step_gmm(X, Y, Z);

d) Generate $10000$ independent samples of size $n$ from the model. For each sample, compute the following:
* The absolute value of the bias of the 2SLS estimator: |$\hat{\beta}^{2SLS}-\beta$|
* The absolute value of the bias of the two-step efficient GMM estimator: |$\hat{\beta}^{GMM}-\beta$|
* The 95% confidence interval for $\beta$ based on the 2SLS estimator and check if the true value $\beta$ is inside.
* The 95% confidence interval for $\beta$ based on the GMM estimator and check if the true value $\beta$ is inside.

In [10]:
function calculate_measures(X, Z, Y, true_β)
    # Calculate 2SLS estimator and its standard error
    β_hat_2sls, se_2sls = two_stage_least_squares(Y, X, Z)
    
    # Calculate efficient GMM estimator and its standard error
    β_hat_gmm, se_gmm = two_step_gmm(X, Y, Z)
    
    # Calculate bias for 2SLS estimator
    bias_2sls = abs(β_hat_2sls - true_β)
    
    # Calculate bias for efficient GMM estimator
    bias_gmm = abs(β_hat_gmm - true_β)
    
    # Calculate 95% confidence interval for beta using 2SLS estimator
    ci_2sls = [β_hat_2sls - 1.96 * se_2sls, β_hat_2sls + 1.96 * se_2sls]
    
    # Check if true value of beta is inside the confidence interval using 2SLS estimator
    is_inside_2sls = Int((true_β >= ci_2sls[1]) && (true_β <= ci_2sls[2]))
    
    # Calculate 95% confidence interval for beta using efficient GMM estimator
    ci_gmm = [β_hat_gmm - 1.96 * se_gmm, β_hat_gmm + 1.96 * se_gmm]
    
    # Check if true value of beta is inside the confidence interval using efficient GMM estimator
    is_inside_gmm = Int((true_β >= ci_gmm[1]) && (true_β <= ci_gmm[2]))
    
    return bias_2sls, bias_gmm, is_inside_2sls, is_inside_gmm, se_2sls, se_gmm
end;


In [11]:
function generate_and_calculate_measures(K, n, true_β, π, γ, ρ, seed)
    bias_2sls_list = []
    bias_gmm_list = []
    se_2sls_list = []
    se_gmm_list = []
    is_inside_2sls_list = []
    is_inside_gmm_list = []
    
    for i in 1:K
        #Y, X, Z = generate_data(n)
        Y, X, Z = generate_observations(true_β, π, γ, ρ, n, seed + i)
        bias_2sls, bias_gmm, is_inside_2sls, is_inside_gmm, se_2sls, se_gmm = calculate_measures(X, Z, Y, true_β)
        
        # bias
        push!(bias_2sls_list, bias_2sls)
        push!(bias_gmm_list, bias_gmm)

        # is inside
        push!(is_inside_2sls_list, is_inside_2sls)
        push!(is_inside_gmm_list, is_inside_gmm)

        # stantard error
        push!(se_2sls_list, se_2sls)
        push!(se_gmm_list, se_gmm)
    end
    
    return bias_2sls_list, bias_gmm_list, is_inside_2sls_list, is_inside_gmm_list, se_2sls_list, se_gmm_list
end;


e) Report the following statistics based on the 2SLS and two-step efficient GMM estimators:

* The average bias across the MC repetitions;
* The average standard error across repetitions;
* The simulated coverage probability for the 95% confidence interval.

In [12]:
bias_2sls_list, bias_gmm_list, is_inside_2sls_list, is_inside_gmm_list, se_2sls_list, se_gmm_list = generate_and_calculate_measures(10000, 500, β, π, γ, ρ, seed);

In [13]:
table_data = ["Simulated coverage" mean(is_inside_2sls_list) mean(is_inside_gmm_list); 
"Average bias" mean(bias_2sls_list) mean(bias_gmm_list);
"Average standard error" mean(se_2sls_list) mean(se_gmm_list)             
]

header=["Statistic" , "2SLS" ,"Two-step efficient GMM"]

pretty_table(table_data;header)

┌────────────────────────┬──────────┬────────────────────────┐
│              Statistic │     2SLS │ Two-step efficient GMM │
├────────────────────────┼──────────┼────────────────────────┤
│     Simulated coverage │    0.967 │                  0.949 │
│           Average bias │ 0.460421 │               0.373939 │
│ Average standard error │ 0.542162 │               0.440369 │
└────────────────────────┴──────────┴────────────────────────┘


f) Compare the two methods (2SLS and two-step efficient GMM) in terms of the statistics listed in the previous part. Which is the preferred method?

Answer: Considering the non-linear form of $U_i$, the two-step efficient GMM performs better than the 2SLS estimator, with a small bias and standard error, on average. Although the simulated coverage is slightly bigger for 2SLS estimator, the GMM is more efficient. Therefore, the GMM simulated coverage seems to provide more precision than the 2SLS one. 

g) Repeat (d)-(f) for $n=100$ and discuss the differences with the simulation results for $n=500$.

Answer: Even with a worse performance when compared to the $n=500$ case, the GMM estimator is superior than the 2SLS estimator. The 2SLS simulated coverage shows how the standard deviation and bias are inflated. 

In [14]:
bias_2sls_list, bias_gmm_list, is_inside_2sls_list, is_inside_gmm_list, se_2sls_list, se_gmm_list = generate_and_calculate_measures(10000, 100, β, π, γ, ρ, seed);

In [15]:
table_data = ["Simulated coverage" mean(is_inside_2sls_list) mean(is_inside_gmm_list); 
"Average bias" mean(bias_2sls_list) mean(bias_gmm_list);
"Average standard error" mean(se_2sls_list) mean(se_gmm_list)             
]

header=["Statistic" , "2SLS" ,"Two-step efficient GMM"]

pretty_table(table_data;header)

┌────────────────────────┬──────────┬────────────────────────┐
│              Statistic │     2SLS │ Two-step efficient GMM │
├────────────────────────┼──────────┼────────────────────────┤
│     Simulated coverage │      1.0 │                 0.5325 │
│           Average bias │ 0.856786 │               0.642517 │
│ Average standard error │  2.11703 │               0.315806 │
└────────────────────────┴──────────┴────────────────────────┘
